## Mediation: Psychological Need Satisfaction → Epistemic Trust → Global AI Acceptance

We next test whether global psychological need satisfaction (TENS_Life_mean) is associated with
overall acceptance of AI mental-health technologies (UTAUT_AI_mean) indirectly through epistemic
trust in mental health technologies (ET_mean), while accounting for demographics, symptoms, stigma,
general AI attitudes, culture, and sample type.

Conceptually, this section evaluates a mediation pathway:

- **Predictor (X):** TENS_Life_mean_c – centered Self-Determination (need satisfaction in life)
- **Mediator (M):** ET_mean_c – centered epistemic trust in mental health technologies
- **Outcome (Y):** UTAUT_AI_mean – global AI acceptance for mental health interventions

We estimate:

1. **Path a:** X → M (Does need satisfaction relate to epistemic trust, net of covariates?)
2. **Paths b and c′:** M and X → Y (Does epistemic trust predict AI acceptance after accounting
   for need satisfaction, and is the direct effect of need satisfaction on AI acceptance still
   non-zero?)
3. **Indirect effect (a × b):** The product of a and b, with a non-parametric bootstrap to obtain a
   confidence interval for the mediated effect.

All predictors are either **mean-centered** (continuous variables) or entered as categorical
contrasts (gender, Country, role_label). We use listwise deletion within the mediation model.


# 0.0. Library and Path Setup

In [2]:
from __future__ import annotations

import warnings
from pathlib import Path
from typing import Dict, List

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf

warnings.filterwarnings("ignore", category=FutureWarning)

PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / "data" / "merged"

MERGED_PATH = DATA_DIR / "merged.csv"

In [3]:
df = pd.read_csv(MERGED_PATH)

print("Raw merged shape:", df.shape)
df.head()

Raw merged shape: (2342, 273)


,AI_use_sum,AIavatar_CO_mean,AIavatar_EOU_mean,AIavatar_HC_mean,AIavatar_HM_mean,AIavatar_PPR_mean,AIavatar_SE_mean,AIavatar_TQE_mean,AIavatar_mean,Accept_avatar,...,gender,jobIT,role_label,would_you_use_1,would_you_use_2,would_you_use_3,would_you_use_4,would_you_use_5,would_you_use_6,would_you_use_7
0,7.0,4.0,4.0,2.75,3.5,2.5,4.0,4.000000,3.535714,3.702381,...,3.0,1.0,therapist,NaN,1.0,NaN,1.0,NaN,1.0,1.0
1,6.0,4.0,4.0,1.75,3.5,2.0,4.0,3.333333,3.226190,3.940476,...,3.0,1.0,therapist,NaN,1.0,1.0,1.0,NaN,1.0,1.0
2,3.0,3.0,3.0,3.00,3.0,3.0,3.0,2.666667,2.952381,2.964286,...,3.0,1.0,therapist,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,2.0,1.5,1.50,1.0,1.0,1.0,1.000000,1.285714,2.571429,...,3.0,1.0,client,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,3.0,2.5,3.0,3.00,3.0,3.0,3.0,3.000000,2.928571,3.000000,...,3.0,1.0,client,NaN,1.0,1.0,1.0,NaN,1.0,NaN


# 1.0. Ensure centered variables already exist

In [5]:
for col in ["TENS_Life_mean", "ET_mean", "age", "GAAIS_mean"]:
    if col in df.columns and f"{col}_c" not in df.columns:
        df[f"{col}_c"] = df[col] - df[col].mean()

# Define variables used in the main mediation model
h4_vars_full = [
    "UTAUT_AI_mean", "TENS_Life_mean_c", "ET_mean_c",
    "age_c", "gender", "PHQ5_mean", "SSRPH_mean", "GAAIS_mean",
    "Country", "role_label"
]

h4_df_full = df[h4_vars_full].dropna().copy()
print(f"H4 sample size (listwise): N = {len(h4_df_full)}")

H4 sample size (listwise): N = 2085


In [6]:
# Quick descriptives for focal variables
print("Descriptives for mediation variables (full-control sample):")
print(h4_df_full[["TENS_Life_mean_c", "ET_mean_c", "UTAUT_AI_mean"]].describe())


Descriptives for mediation variables (full-control sample):
       TENS_Life_mean_c    ET_mean_c  UTAUT_AI_mean
count       2085.000000  2085.000000    2085.000000
mean           0.009787     0.000636       4.788588
std            1.020626     0.860827       1.384876
min           -3.758704    -3.272937       1.115385
25%           -0.758704    -0.606271       3.448718
50%            0.116296    -0.139604       5.000000
75%            0.741296     0.527063       6.037037
max            2.241296     2.727063       7.814815


In [8]:
# Correlations among X, M, Y (for a quick sense of direction)
print("Pearson correlations among TENS_Life_mean_c, ET_mean_c, UTAUT_AI_mean:")
print(h4_df_full[["TENS_Life_mean_c", "ET_mean_c", "UTAUT_AI_mean"]].corr())

Pearson correlations among TENS_Life_mean_c, ET_mean_c, UTAUT_AI_mean:
                  TENS_Life_mean_c  ET_mean_c  UTAUT_AI_mean
TENS_Life_mean_c          1.000000  -0.139005       0.165927
ET_mean_c                -0.139005   1.000000       0.036175
UTAUT_AI_mean             0.165927   0.036175       1.000000


# 2.0. Model Specification (Full-Control)

We first estimate the mediation pathway **controlling for the same covariates used in earlier hypothesis tests (H1–H3)** to keep the adjustment set consistent:

- **Covariates:** age_c, gender, PHQ5_mean (depressive symptoms),
  SSRPH_mean (self-stigma), GAAIS_mean (general AI attitudes),
  Country (China vs USA), and role_label (client vs therapist vs unknown).

Thus:

- **Path a model (X → M):**

  `ET_mean_c ~ TENS_Life_mean_c + age_c + C(gender) + PHQ5_mean + SSRPH_mean + GAAIS_mean + C(Country) + C(role_label)`

- **Path b + c′ model (M and X → Y):**

  `UTAUT_AI_mean ~ TENS_Life_mean_c + ET_mean_c + age_c + C(gender) + PHQ5_mean + SSRPH_mean + GAAIS_mean + C(Country) + C(role_label)`

We estimate these models with OLS using listwise deletion on the full set of variables.

In [12]:
print("=== 6.5.1. Full-Control Mediation: TENS → ET → UTAUT ===")
print(f"H4 full-control sample size (listwise): N = {len(h4_df_full)}")

# Path a: TENS → ET (full-control)
a_model_full = smf.ols(
    formula=(
        "ET_mean_c ~ TENS_Life_mean_c "
        "+ age_c + C(gender) + PHQ5_mean + SSRPH_mean + GAAIS_mean "
        "+ C(Country) + C(role_label)"
    ),
    data=h4_df_full
).fit()

print("H4 Path a (TENS_Life_mean_c → ET_mean_c; full-control):")
print(a_model_full.summary().tables[1])

=== 6.5.1. Full-Control Mediation: TENS → ET → UTAUT ===
H4 full-control sample size (listwise): N = 2085
H4 Path a (TENS_Life_mean_c → ET_mean_c; full-control):
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                     -1.2601      0.132     -9.522      0.000      -1.520      -1.001
C(gender)[T.2.0]               0.0101      0.034      0.295      0.768      -0.057       0.077
C(gender)[T.3.0]              -0.0311      0.116     -0.269      0.788      -0.258       0.195
C(gender)[T.4.0]              -0.4040      0.283     -1.428      0.153      -0.959       0.151
C(Country)[T.USA]              0.0423      0.036      1.180      0.238      -0.028       0.113
C(role_label)[T.therapist]     0.1026      0.068      1.502      0.133      -0.031       0.237
C(role_label)[T.unknown]       0.0423      0.036      1.180      0.238      -0

In [11]:
# Path b + c': ET + TENS → UTAUT (full-control)
b_model_full = smf.ols(
    formula=(
        "UTAUT_AI_mean ~ TENS_Life_mean_c + ET_mean_c "
        "+ age_c + C(gender) + PHQ5_mean + SSRPH_mean + GAAIS_mean "
        "+ C(Country) + C(role_label)"
    ),
    data=h4_df_full
).fit()

print("H4 Path b + c' (ET_mean_c and TENS_Life_mean_c → UTAUT_AI_mean; full-control):")
print(b_model_full.summary().tables[1])

H4 Path b + c' (ET_mean_c and TENS_Life_mean_c → UTAUT_AI_mean; full-control):
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      1.8268      0.211      8.648      0.000       1.413       2.241
C(gender)[T.2.0]              -0.0931      0.053     -1.746      0.081      -0.198       0.011
C(gender)[T.3.0]               0.0809      0.181      0.448      0.654      -0.273       0.435
C(gender)[T.4.0]               0.5582      0.442      1.262      0.207      -0.309       1.426
C(Country)[T.USA]              1.1012      0.056     19.665      0.000       0.991       1.211
C(role_label)[T.therapist]     0.0722      0.107      0.676      0.499      -0.137       0.282
C(role_label)[T.unknown]       1.1012      0.056     19.665      0.000       0.991       1.211
TENS_Life_mean_c               0.0933      0.027      3.470      0

# Interpretation of Full-Control Mediation

**Path a (TENS_Life_mean_c → ET_mean_c).** In the full-control model, the coefficient for
TENS_Life_mean_c is small and statistically non-significant:

- `b_a ≈ -0.023`, p ≈ .18

After adjusting for age, gender, depressive symptoms, stigma, general AI attitudes, country, and
sample type, higher psychological need satisfaction is **not reliably associated** with higher
epistemic trust in AI mental-health tools. The point estimate is slightly negative, but the
confidence interval crosses zero.

**Path b (ET_mean_c → UTAUT_AI_mean).** In contrast, epistemic trust is a robust positive predictor
of global AI acceptance:

- `b_b ≈ 0.20`, p < .001

Holding TENS_Life_mean_c and covariates constant, participants who report higher trust in AI
mental-health technologies also report **higher overall acceptance** of AI in mental health (UTAUT).

**Direct effect c′ (TENS_Life_mean_c → UTAUT_AI_mean).** Need satisfaction retains a positive
association with AI acceptance:

- `b_c' ≈ 0.09`, p ≈ .001

Even after controlling for epistemic trust and other covariates, higher psychological need
satisfaction is linked to **more positive global attitudes** toward AI mental-health interventions.

Taken together, the full-control model suggests:

1. Epistemic trust is a strong proximal predictor of AI acceptance.
2. Psychological need satisfaction has a **direct positive association** with AI acceptance,
   independent of trust and covariates.
3. The X → M link (need satisfaction → epistemic trust) is weak in this fully adjusted model, so any
   mediated pathway is expected to be small in magnitude.

# 3.0. Simplified Mediation + Bootstrap Rationale

### Parsimonious Mediation Model and Bootstrap

To provide a more transparent estimate of the indirect effect and reduce over-adjustment, we follow a
simplified specification that only adjusts for **demographics and culture** (age_c, gender, Country)
while focusing on the core mediation pathway. This is consistent with the code used for the
bootstrap analysis.

Simplified models:

- **Path a (parsimonious):**

  `ET_mean_c ~ TENS_Life_mean_c + age_c + C(gender) + C(Country)`

- **Paths b and c′ (parsimonious):**

  `UTAUT_AI_mean ~ TENS_Life_mean_c + ET_mean_c + age_c + C(gender) + C(Country)`

We then compute:

- The **point estimate of the indirect effect** `a × b`.
- A **non-parametric bootstrap** (5,000 resamples) to obtain a bias-corrected
  95% confidence interval for `a × b`.

This complements the full-control model by offering a more interpretable mediation estimate with a
leaner set of covariates.


## 3.1. Simplified Mediation and a×b

### 3.1.1. Parsimonious mediation model

In [14]:
h4_vars_simple = [
    "TENS_Life_mean_c", "ET_mean_c",
    "UTAUT_AI_mean", "age_c", "gender", "Country"
]

h4_df_simple = df[h4_vars_simple].dropna().copy()
print(f"H4 (parsimonious) sample size (listwise): N = {len(h4_df_simple)}")

H4 (parsimonious) sample size (listwise): N = 2096


### 3.1.2. Path a: TENS → ET (parsimonious)

In [15]:
a_model_simple = smf.ols(
    formula="ET_mean_c ~ TENS_Life_mean_c + age_c + C(gender) + C(Country)",
    data=h4_df_simple
).fit()

print("H4 Path a (TENS_Life_mean_c → ET_mean_c; parsimonious):")
print(a_model_simple.summary().tables[1])

H4 Path a (TENS_Life_mean_c → ET_mean_c; parsimonious):
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.1896      0.052      3.632      0.000       0.087       0.292
C(gender)[T.2.0]      0.0599      0.037      1.633      0.103      -0.012       0.132
C(gender)[T.3.0]      0.0340      0.126      0.269      0.788      -0.214       0.282
C(gender)[T.4.0]     -0.3645      0.311     -1.174      0.241      -0.974       0.245
C(Country)[T.USA]    -0.2864      0.059     -4.870      0.000      -0.402      -0.171
TENS_Life_mean_c     -0.0549      0.018     -3.031      0.002      -0.090      -0.019
age_c                -0.0083      0.001     -5.692      0.000      -0.011      -0.005


### 3.1.3. Path b + c': ET + TENS → UTAUT (parsimonious)

In [16]:
b_model_simple = smf.ols(
    formula="UTAUT_AI_mean ~ TENS_Life_mean_c + ET_mean_c + age_c + C(gender) + C(Country)",
    data=h4_df_simple
).fit()

print("H4 Path b + c' (ET_mean_c and TENS_Life_mean_c → UTAUT_AI_mean; parsimonious):")
print(b_model_simple.summary().tables[1])

H4 Path b + c' (ET_mean_c and TENS_Life_mean_c → UTAUT_AI_mean; parsimonious):
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             3.1564      0.075     41.866      0.000       3.009       3.304
C(gender)[T.2.0]     -0.0402      0.053     -0.761      0.447      -0.144       0.063
C(gender)[T.3.0]      0.0711      0.182      0.391      0.696      -0.286       0.428
C(gender)[T.4.0]      0.5395      0.447      1.207      0.228      -0.337       1.416
C(Country)[T.USA]     2.1359      0.085     25.092      0.000       1.969       2.303
TENS_Life_mean_c      0.1197      0.026      4.585      0.000       0.068       0.171
ET_mean_c             0.2507      0.031      7.962      0.000       0.189       0.312
age_c                -0.0204      0.002     -9.676      0.000      -0.025      -0.016


### 3.1.4. Point estimate of indirect effect

In [18]:
a_coef = a_model_simple.params["TENS_Life_mean_c"]
b_coef = b_model_simple.params["ET_mean_c"]
print(f"Point estimate of indirect effect (a * b) = {a_coef * b_coef:.4f}")

Point estimate of indirect effect (a * b) = -0.0138


# 4.0. Bootstrap for Indirect Effect

## 4.1. Bootstrap for indirect effect a × b

In [19]:
n_boot = 5000
boot_indirect = []
n = len(h4_df_simple)

for _ in range(n_boot):
    sample_idx = np.random.choice(n, size=n, replace=True)
    sample = h4_df_simple.iloc[sample_idx]

    # Path a in bootstrap sample
    a_m = smf.ols(
        formula="ET_mean_c ~ TENS_Life_mean_c + age_c + C(gender) + C(Country)",
        data=sample
    ).fit()

    # Path b + c' in bootstrap sample
    b_m = smf.ols(
        formula="UTAUT_AI_mean ~ TENS_Life_mean_c + ET_mean_c + age_c + C(gender) + C(Country)",
        data=sample
    ).fit()

    a_b = a_m.params.get("TENS_Life_mean_c", np.nan)
    b_b = b_m.params.get("ET_mean_c", np.nan)

    if not np.isnan(a_b) and not np.isnan(b_b):
        boot_indirect.append(a_b * b_b)

boot_indirect = np.array(boot_indirect)
indirect_mean = np.mean(boot_indirect)
ci_lower, ci_upper = np.percentile(boot_indirect, [2.5, 97.5])

In [20]:
print(f"Bootstrapped indirect effect (a*b): mean = {indirect_mean:.4f}")
print(f"95% CI: [{ci_lower:.4f}, {ci_upper:.4f}]")
print(f"Number of successful bootstrap samples: {len(boot_indirect)} / {n_boot}")

Bootstrapped indirect effect (a*b): mean = -0.0139
95% CI: [-0.0262, -0.0024]
Number of successful bootstrap samples: 5000 / 5000


# Final H4 Interpretation & Reviewer-Friendly Summary

## Summary of H4 Mediation Findings

Across both the full-control and parsimonious specifications, the mediation pattern is consistent and
nuanced:

1. **X → M (Need satisfaction → Epistemic trust).**

   - In the parsimonious model (controlling for age, gender, Country), the path from
     TENS_Life_mean_c to ET_mean_c is **negative and statistically significant**
     (b ≈ −0.055, p ≈ .002).
   - In the full-control model (adding symptoms, stigma, and general AI attitudes), this path
     becomes small and non-significant (b ≈ −0.023, p ≈ .18), suggesting that once we account
     for psychological distress, stigma, and AI attitudes, the direct link from need
     satisfaction to epistemic trust is weak.

2. **M → Y (Epistemic trust → AI acceptance).**

   - In all specifications, ET_mean_c is a **strong, positive predictor** of UTAUT_AI_mean
     (b ≈ 0.20–0.25, p < .001). Participants who express greater trust in AI mental-health
     technologies consistently report higher global acceptance of AI in mental health.

3. **Direct effect c′ (Need satisfaction → AI acceptance).**

   - TENS_Life_mean_c has a **positive direct association** with UTAUT_AI_mean even after
     including ET_mean_c and covariates (b ≈ 0.09–0.12, p < .01). This suggests that people who
     experience their psychological needs as more satisfied tend to view AI mental-health tools
     more favorably overall, beyond any influence of epistemic trust.

4. **Indirect effect (a × b).**

   - In the parsimonious mediation model, the product of paths a and b is **negative and small**:
     a × b ≈ −0.0138.
   - The non-parametric bootstrap confirms that this indirect effect is **statistically
     distinguishable from zero**, with a 95% CI approximately
     [−0.0264, −0.0020], based on 5,000 resamples.

5. **Interpretation and magnitude.**

   - The negative indirect effect, combined with a positive direct effect, indicates a subtle
     **suppression pattern** rather than a classic “need satisfaction → trust → acceptance”
     mechanism. On average, higher need satisfaction is directly associated with more positive
     AI acceptance, but is also (slightly) associated with lower epistemic trust in AI under
     the parsimonious model. Because epistemic trust itself strongly increases AI acceptance,
     this small negative a path produces a **tiny negative mediated component** that slightly
     attenuates the overall positive relationship between need satisfaction and AI acceptance.
   - In practical terms, the magnitude of the indirect effect is **very small** relative to the
     scale of UTAUT_AI_mean and to the direct effects. Thus, the data do **not** support a
     strong claim that epistemic trust is the primary mechanism by which need satisfaction
     translates into AI acceptance. Instead, trust operates more as an **independent proximal
     predictor** of acceptance, alongside psychological need satisfaction, distress, stigma,
     and general AI attitudes.

Overall, H4 suggests that:

- **Epistemic trust is a robust and important correlate of global AI acceptance**, and
- **Self-Determination remains directly linked to AI acceptance**, but the mediation pathway
  through epistemic trust is small, complex in sign, and sensitive to covariate adjustment.

These results align with a picture in which **need satisfaction and epistemic trust are related but
partially distinct contributors** to how participants evaluate AI mental-health technologies, rather
than a simple unidirectional chain from needs → trust → acceptance.